참고문헌  
\[1\] [A Complete Guide to BERT with Code](https://towardsdatascience.com/a-complete-guide-to-bert-with-code-9f87602e4a11/)

# Fine-Tuning BERT for Sentiment Analysis

## 3.1 -Load and Preprocess a Fine-Tunning Dataset

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../Datasets/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


이전의 NLP 모델들과는 달리, BERT와 같은 트랜스포머 기반 모델은 전처리가 거의 필요하지 않다.  
불용어(stop words)나 구두점(punctuation)을 제거하는 단계는 경우에 따라 오히려 역효과를 낳을 수 있다.  
이러한 요소들이 입력 문장을 이해하는 데 유용한 문맥 정보를 `BERT`에게 제공하기 때문이다.  
  
그럼에도 불구하고, 텍스트에 포맷 문제나 원치 않는 특수 문자 등이 있는지 확인하는 작업은 여전히 중요하다.  
전반적으로 IMDb 데이터셋은 꽤 깨끗한 편이지만, 스크래핑 과정에서 생긴 일부 흔적들이 남아 있는 것으로 보인다.  
>예를 들어:  
HTML 줄바꿈 태그 (\<br />)  
불필요한 공백(whitespace) 등  

이러한 요소들은 제거해주는 것이 좋다.

In [3]:
# Remove the break tags (<br />)
df['review_cleaned'] = df['review'].apply(lambda x: x.replace('<br />', ''))

# Remove unnecessary whitespace
df['review_cleaned'] = df['review_cleaned'].replace('s+', ' ', regex=True)

# Compare 72 characters of the second review before and after cleaning
print('Before cleaning:')
print(df.iloc[1]['review'][0:72])

print('nAfter cleaning:')
print(df.iloc[1]['review_cleaned'][0:72])

Before cleaning:
A wonderful little production. <br /><br />The filming technique is very
nAfter cleaning:
A wonderful little production. The filming technique i  very una uming- 


### Encode the Sentiment:  
전처리의 마지막 단계는 각 리뷰의 감정을 부정일 경우 0, 긍정일 경우 1로 인코딩하는 것이다.  
이러한 라벨은 파인튜닝 과정에서 분류 헤드(classification head)를 학습시키는 데 사용된다.  

In [4]:
df['sentiment_encoded'] = df['sentiment'].apply(lambda x: 0 if x == 'negative' else 1)
df.head()

,review,sentiment,review_cleaned,sentiment_encoded
0,One of the other reviewers has mentioned that ...,positive,One of the other reviewer ha mentioned that ...,1
1,A wonderful little production. <br /><br />The...,positive,A wonderful little production. The filming tec...,1
2,I thought this was a wonderful way to spend ti...,positive,I thought thi wa a wonderful way to pend ti...,1
3,Basically there's a family where a little boy ...,negative,Ba ically there' a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"Petter Mattei' ""Love in the Time of Money"" i ...",1


## 3.2 – 파인튜닝 데이터 토크나이즈(Tokenize)
전처리가 완료되면, 파인튜닝 데이터에 대해 토크나이즈(`tokenization`) 과정을 수행할 수 있다.  
이 과정에서는 다음과 같은 작업이 이루어진다:  

리뷰 텍스트를 개별 토큰(token) 으로 분할  
[CLS]와 [SEP] 같은 특수 토큰 추가  

시퀀스 길이를 맞추기 위한 패딩 처리  

모델마다 요구하는 토크나이징 방식이 다르기 때문에, 해당 모델에 맞는 적절한 토크나이저를 선택하는 것이 중요하다.  
예를 들어, GPT는 [CLS] 및 [SEP] 토큰을 사용하지 않지만, BERT는 이를 사용한다.  

이번 예제에서는 `Hugging Face`의 `transformers` 라이브러리에서 제공하는 `BertTokenizer` 클래스를 사용한다.  
이 토크나이저는 `BERT` 계열 모델과 함께 사용하도록 설계되어 있다.  
토크나이징 작동 방식에 대한 보다 자세한 설명은 이 시리즈의 Part 1을 참고하면 된다.

transformers 라이브러리의 토크나이저 클래스는 from_pretrained 메서드를 사용하여  
**사전학습된 토크나이저 모델을 손쉽게 생성**할 수 있다.  
이를 사용하는 방법은 다음과 같다:  

1. 토크나이저 클래스를 import 및 인스턴스화  
2. from_pretrained 메서드를 호출  
3. Hugging Face 모델 저장소에 있는 토크나이저 이름(문자열 형태)을 인자로 전달
  
또는, 로컬에 저장된 단어 집합(vocabulary) 파일이 포함된 디렉터리 경로를 전달하는 것도 가능하다 [9].  

---  

이번 예제에서는 Hugging Face 모델 저장소에서 제공하는 사전학습된 토크나이저를 사용할 것이며,  
BERT 관련으로는 다음과 같은 네 가지 주요 옵션이 있다.   
이들은 모두 Google의 사전학습 BERT 토크나이저의 어휘를 기반으로 한다:  

- bert-base-uncased  
→ BERT Base 모델용, 대소문자 구분 안 함 (예: Cat과 cat을 동일하게 처리)

- bert-base-cased  
→ BERT Base 모델용, 대소문자 구분함 (예: Cat과 cat을 다르게 처리)

- bert-large-uncased  
→ BERT Large 모델용, 대소문자 구분 안 함

- bert-large-cased  
→ BERT Large 모델용, 대소문자 구분함

`BERT Base`와 `BERT Large`는 **동일한 어휘 집합(vocabulary)** 을 사용하므로,  
`bert-base-uncased`와 `bert-large-uncased`는 실질적으로 차이가 없고,  
`bert-base-cased`와 `bert-large-cased`도 동일한 어휘를 사용한다.  

그러나 다른 모델들에서는 동일하지 않을 수 있으므로, 확실하지 않다면 모델과 토크나이저의  
크기를 일치시켜 사용하는 것이 가장 안전하다.


### 대소문자 구분(cased vs uncased)을 언제 사용할까?
(When to Use cased vs uncased:)  

cased 모델과 uncased 모델 중 어떤 것을 사용할지는 데이터셋의 특성에 따라 달라진다.  
예를 들어, IMDb 데이터셋은 인터넷 사용자들이 작성한 리뷰로 구성되어 있다.  
이들은 대소문자 사용에 일관성이 없을 수 있다.  

어떤 사용자는 문장의 첫 글자에서 대문자를 생략하기도 하고,  
어떤 사용자는 **강조나 감정 표현(흥분, 분노 등)**을 위해 대문자를 과하게 사용하기도 한다.  
이러한 이유로 우리는 대소문자를 무시(uncased) 하기로 결정하고,  
bert-base-uncased 토크나이저 모델을 사용할 것이다.  

하지만, 경우에 따라 대소문자를 구분하는(cased) 것이 성능 향상에 도움이 되기도 한다.  
대표적인 예로는 개체명 인식(Named Entity Recognition, NER) 과제가 있다.  
이 과제에서는 입력 텍스트 내에서 사람, 조직, 장소 등의 **고유 명사(Entity)** 를 식별하는 것이 목표인데,  
이 경우 대문자의 존재 여부는 해당 단어가 사람 이름인지, 장소 이름인지 등을 판별하는 데 매우 유용하다.  

따라서, 이러한 상황에서는 bert-base-cased 모델을 사용하는 것이 더 적합할 수 있다.  

In [5]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
print(tokenizer)

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


### **Encoding Process: Converting Text to Tokens to Token IDs**

제 전처리된 파인튜닝 데이터를 인코딩할 차례다.  
이 과정에서는 각 리뷰가 **토큰 ID로 이루어진 텐서(tensor)**로 변환된다.  

예를 들어, 리뷰 "I liked this movie" 는 다음 단계를 통해 인코딩된다:  

1. 소문자 변환  
→ 우리가 bert-base-uncased를 사용하고 있으므로, 입력 문장은 모두 소문자로 변환된다.  

2. 토큰 분할  
→ BERT 어휘(bert-base-uncased의 vocabulary)에 따라 문장을 토큰 단위로 분할:  
['i', 'liked', 'this', 'movie']  

3. 특수 토큰 추가  
→ BERT에서 기대하는 특수 토큰을 추가:  
['[CLS]', 'i', 'liked', 'this', 'movie', '[SEP]']  

4. 토큰을 토큰 ID로 변환  
→ 각 토큰을 BERT 어휘에 따라 정수형 ID로 매핑 (예: [CLS] → 101, i → 1045 등)  
---
이 전체 인코딩 과정은 `BertTokenizer` 클래스의 `encode` 메서드를 통해 수행할 수 있다.  
이 메서드는 텍스트를 인코딩하여 토큰 ID 텐서를 반환하며, 다음 중 하나의 형식으로 반환 가능하다:  

- PyTorch 텐서 (pt)
- TensorFlow 텐서 (tf)
- NumPy 배열 (np)

반환 형식은 return_tensors 인자를 통해 지정할 수 있다.  

> 💡 참고: Hugging Face에서는 Token ID를 종종 Input ID라고 부르기도 하며, 두 용어는 혼용되어 사용된다.

In [8]:
# Encode a sample input sentence
sample_sentence = 'I liked this movie'
token_ids = tokenizer.encode(sample_sentence, return_tensors='np')[0]
print(f'Token IDs: {token_ids}')

# Convert the token IDs back to tokens to reveal the special tokens added
tokens = tokenizer.convert_ids_to_tokens(token_ids)
print(f'Tokens   : {tokens}')

Token IDs: [ 101 1045 4669 2023 3185  102]
Tokens   : ['[CLS]', 'i', 'liked', 'this', 'movie', '[SEP]']


### **Truncation and Padding:**

BERT Base와 BERT Large는 입력 시퀀스가 정확히 512개 토큰일 때 처리하도록 설계되어 있다.  
그렇다면 입력 시퀀스가 이 길이를 초과하거나 부족할 경우에는 어떻게 해야 할까?  
정답은 바로 **잘라내기(truncation)** 와 **패딩(padding)** 이다!  

**잘라내기 (Truncation):**    
잘라내기는 지정된 길이를 초과하는 토큰들을 단순히 잘라서 제거하는 방식이다.  
encode 메서드에서 truncation=True로 설정하고, max_length 인자를 지정하면  
모든 인코딩된 시퀀스에 대해 길이 제한을 강제할 수 있다.  

이 데이터셋의 일부 리뷰는 512개 토큰 제한을 초과하므로,  
가장 많은 텍스트를 반영할 수 있도록 max_length=512로 설정한다.  
만약 모든 리뷰가 512 토큰을 넘지 않는다면, max_length는 생략해도 되고,  
기본적으로 **모델의 최대 입력 길이(512)**가 자동으로 적용된다.  

또는, 학습 시간을 줄이기 위해 512보다 짧은 길이를 지정하는 것도 가능하다.  
다만, 이 경우 모델 성능이 다소 희생될 수 있음에 유의해야 한다.  

**패딩 (Padding):**    
대부분의 리뷰는 512 토큰보다 짧기 때문에,  
이런 경우에는 [PAD] 토큰을 추가하여 시퀀스를 512 토큰까지 확장한다.  
이를 위해 padding='max_length' 옵션을 설정하면 된다.  

자세한 내용은 Hugging Face의 encode 메서드 관련 문서를 참조하자 [10].  

In [12]:
review = df['review_cleaned'].iloc[0]

print("review  example.");
print(review)

print("----------------------")

token_ids = tokenizer.encode(
    review,
    max_length = 512,
    padding = 'max_length',
    truncation = True,
    return_tensors = 'pt')

print(token_ids)

review  example.
One of the other reviewer  ha  mentioned that after watching ju t 1 Oz epi ode you'll be hooked. They are right, a  thi  i  exactly what happened with me.The fir t thing that  truck me about Oz wa  it  brutality and unflinching  cene  of violence, which  et in right from the word GO. Tru t me, thi  i  not a  how for the faint hearted or timid. Thi   how pull  no punche  with regard  to drug ,  ex or violence. It  i  hardcore, in the cla ic u e of the word.It i  called OZ a  that i  the nickname given to the O wald Maximum Security State Penitentary. It focu e  mainly on Emerald City, an experimental  ection of the pri on where all the cell  have gla  front  and face inward ,  o privacy i  not high on the agenda. Em City i  home to many..Aryan , Mu lim , gang ta , Latino , Chri tian , Italian , Iri h and more.... o  cuffle , death  tare , dodgy dealing  and  hady agreement  are never far away.I would  ay the main appeal of the  how i  due to the fact that it goe  where 

### `encode_plus`와 함께 Attention Mask 사용하기

앞서 살펴본 예제에서는 데이터셋의 첫 번째 리뷰가 인코딩된 결과, `119`개의 패딩 토큰을 포함하고 있다.  
이 상태 그대로 파인튜닝에 사용하면, BERT가 패딩 토큰에까지 주의를 기울이게 되어  
성능 저하로 이어질 수 있다.  

이를 해결하기 위해 우리는 **어텐션 마스크(attention mask)** 를 적용할 수 있다.  
어텐션 마스크는 BERT에게 입력에서 특정 토큰(이 경우 패딩 토큰)을 무시하라고 지시하는 역할을 한다.    

-----

이를 구현하려면 기존의 encode 메서드 대신,  
encode_plus 메서드를 사용하도록 코드를 수정하면 된다.  

encode_plus는 Hugging Face에서 **배치 인코더(Batch Encoder)** 라고 불리는  
딕셔너리 형태의 값을 반환하며, 다음과 같은 키를 포함한다:  

- **input_ids**:  
→ encode 메서드와 동일하게 토큰 ID들을 반환  

- **token_type_ids**:  
→ 문장 쌍 과제(Natural Language Inference, Next Sentence Prediction 등)에서
문장 A(값 = 0)와 문장 B(값 = 1)를 구분하는 세그먼트 ID

- **attention_mask**:  
→ 0과 1로 이루어진 리스트
→ 0은 해당 토큰이 어텐션에서 무시되어야 함을 의미 (예: 패딩 토큰)  
→ 1은 해당 토큰이 어텐션에서 사용되어야 함을 의미  

이 어텐션 마스크를 통해 BERT는 실제 의미 있는 토큰에만 집중할 수 있게 되며,  
모델의 성능이 유지되거나 향상될 수 있다.  


In [18]:
review = df['review_cleaned'].iloc[0]

batch_encoder = tokenizer.encode_plus(
    review,
    max_length = 512,
    padding = 'max_length',
    truncation = True,
    return_tensors = 'pt')

print('Batch encoder keys:')
print(batch_encoder.keys())

print('\nAttention mask:')
print(batch_encoder['attention_mask'])

print('\ninput_ids:')
print(batch_encoder['input_ids'])

print('\ntoken_type_ids:')
print(batch_encoder['token_type_ids'])

Batch encoder keys:
dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

Attention mask:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1,

### **Encode All Reviews:**  
토크나이징 단계의 마지막 단계는,  데이터셋에 포함된 모든 리뷰를 인코딩하고    
그 결과로 얻은 토큰 ID와 어텐션 마스크를 텐서(tensor) 형식으로 저장하는 것이다.  

In [19]:
import torch

token_ids = []
attention_masks = []

# Encode each review
for review in df['review_cleaned']:
    batch_encoder = tokenizer.encode_plus(
        review,
        max_length = 512,
        padding = 'max_length',
        truncation = True,
        return_tensors = 'pt')

    token_ids.append(batch_encoder['input_ids'])
    attention_masks.append(batch_encoder['attention_mask'])

# Convert token IDs and attention mask lists to PyTorch tensors
token_ids = torch.cat(token_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

In [22]:
token_ids.shape
attention_masks.shape

torch.Size([50000, 512])